# Running the morphologically cell shape analysis notebook

## After we get every cell object as well as the dividing cells, we should start to calculate the cell volume, cell surface and cell-cell contact surface for the further physical-biologcial statistical analysis.

## ONLY FOR COMPLETE LINEAGE (FROM TIME POINT 1)

### morphological calculating configuration

In [1]:
import os
import pickle
from copy import deepcopy
import nibabel as nib
import numpy as np
import pandas as pd
import open3d as o3d
import multiprocessing as mp
import matplotlib.pyplot as plt
from scipy import ndimage
from tqdm import tqdm

from utils.cell_func import get_cell_name_affine_table
from utils.general_func import load_nitf2_img
from utils.shape_model import generate_alpha_shape

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
stat_data_path=r'./middle_output'

def calculate_cell_surface_and_contact_points(config_arg, is_debug=False):
    embryo_name = config_arg['embryo_name']
    is_calculate_cell_mesh = config_arg.get('is_calculate_cell_mesh',None)
    is_calculate_contact_file = config_arg.get('is_calculate_contact_file',None)
    showCellMesh = config_arg['showCellMesh']
    showCellContact = config_arg.get('showCellContact',None)
    time_point = config_arg['time_point']
    path_embryo = config_arg.get('path_embryo', None)

    if path_embryo is None:
        raise Exception("Sorry, no path_embryo given!")

    # ------------------------calculate surface points using dialation for each cell --------------------
    frame_this_embryo = str(time_point).zfill(3)
    file_name = embryo_name + '_' + frame_this_embryo + '_segCell.nii.gz'

    # ------------if contact file exists, finish this embryo---------------
    # contact_saving_path = os.path.join(my_config.data_cell_mesh_and_contact, 'stat', embryo_name,
    #                                    embryo_name+'_'+file_name.split('.')[0] + '_contact.txt')
    # # ===============very important line===================
    # if os.path.exists(contact_saving_path):
    #     print(contact_saving_path,' existed')
    #     return 0
    # # =====================================================

    volume = nib.load(os.path.join(path_embryo, file_name)).get_fdata().astype(int).transpose([2, 1, 0])
    if is_debug:
        print(np.unique(volume, return_counts=True))
    # this_img = load_nitf2_img()

    # volume = this_img.get_fdata().astype(int)
    # -----!!!!!!!!get cell edge boundary!!!!--------------
    cell_mask = volume != 0
    boundary_mask = (cell_mask == 0) & ndimage.binary_dilation(cell_mask)
    [x_bound, y_bound, z_bound] = np.nonzero(boundary_mask)
    boundary_elements = []

    # find boundar/points between cells
    for (x, y, z) in zip(x_bound, y_bound, z_bound):
        neighbors = volume[np.ix_(range(x - 1, x + 2), range(y - 1, y + 2), range(z - 1, z + 2))]
        neighbor_labels = list(np.unique(neighbors))
        neighbor_labels.remove(0)
        if len(neighbor_labels) == 2:  # contact between two cells
            boundary_elements.append(neighbor_labels)
    # cell contact pairs
    cell_contact_pairs = list(np.unique(np.array(boundary_elements), axis=0))
    cell_conatact_pair_renew = []
    contact_points_dict = {}
    contact_area_dict = {}

    for (label1, label2) in cell_contact_pairs:
        contact_mask = np.logical_and(ndimage.binary_dilation(volume == label1),
                                      ndimage.binary_dilation(volume == label2))
        contact_mask = np.logical_and(contact_mask, boundary_mask)
        if contact_mask.sum() > 2:

            cell_conatact_pair_renew.append((label1, label2))
            str_key = str(label1) + '_' + str(label2)
            contact_area_dict[str_key] = 0

            point_position_x, point_position_y, point_position_z = np.where(contact_mask == True)

            contact_points_list = []
            for i in range(len(point_position_x)):
                contact_points_list.append([point_position_x[i], point_position_y[i], point_position_z[i]])
            # print(str_key)
            contact_points_dict[str_key] = contact_points_list
            if is_debug:
                print('contact', str_key, len(contact_points_list))

    if is_debug:
        print('volume info', np.unique(volume, return_counts=True))
    cell_list = np.unique(volume)
    contact_mesh_dict = {}
    showing_record = []

    volume_dict = {}
    surface_dict = {}
    contact_dict = {}

    if is_debug:
        print('configuration', config_arg)
        print('cell list ', cell_list)

    # if not is_calculate_contact_file:
    #     print('loading ', contact_saving_path)
    #     with open(contact_saving_path,'rb') as handle:
    #         contact_mesh_dict = pickle.load(handle)

    # print(cell_list)
    weight_surface = 1.2031
    count_ratio_tmp = []
    for cell_key in cell_list:
        if cell_key != 0:
            cell_mask = np.logical_xor(ndimage.binary_dilation(volume == cell_key), (volume == cell_key))
            # if is_debug:
            #     print('-------',cell_key,'---------')
            #     print('surface num',(cell_mask==1).sum())
            #     print('inside sum',(volume == cell_key).sum())
            if (cell_mask == 1).sum() > 15000:
                volume_dict[cell_key] = (volume == cell_key).sum()
                surface_dict[cell_key] = cell_mask.sum() * weight_surface  # 1.2031... is derived by other papers
                irregularity = surface_dict[cell_key] ** (1 / 2) / volume_dict[cell_key] ** (1 / 3)
                if is_debug:
                    print('irregularity   ', irregularity)
                if irregularity < 2.199085:
                    print('impossible small surface', time_point, cell_key)
                for (cell1, cell2) in cell_conatact_pair_renew:
                    idx = str(cell1) + '_' + str(cell2)
                    # idx_test=
                    if cell_key not in (cell1, cell2) or idx in contact_dict.keys():
                        continue
                    # --------------------contact-----------------------------------------
                    contact_dict[idx] = len(contact_points_dict[idx]) * weight_surface
            else:
                if is_debug:
                    print(cell_key, 'surface point num', (cell_mask == 1).sum(), ' inside point num(dia)',
                          (ndimage.binary_dilation(volume == cell_key) == 1).sum())
                # ------------saving cell mesh---------------------

                tuple_tmp = np.where(ndimage.binary_dilation(volume == cell_key) == 1)
                # print(len(tuple_tmp))
                sphere_list = np.concatenate(
                    (tuple_tmp[0][:, None], tuple_tmp[1][:, None], tuple_tmp[2][:, None]), axis=1)
                adjusted_rate = 0.01
                sphere_list_adjusted = sphere_list.astype(float) + np.random.uniform(0, adjusted_rate,
                                                                                     (len(tuple_tmp[0]), 3))
                m_mesh = generate_alpha_shape(sphere_list_adjusted, displaying=showCellMesh)

                alpha_v = 1

                if not m_mesh.is_watertight():
                    for i in range(10):
                        alpha_v = alpha_v + i * 0.1
                        sphere_list_adjusted = sphere_list.astype(float) + np.random.uniform(0, adjusted_rate * (i + 1),
                                                                                             (len(tuple_tmp[0]), 3))
                        m_mesh = generate_alpha_shape(sphere_list_adjusted, alpha_value=alpha_v,
                                                      displaying=showCellMesh)
                        if is_debug:
                            print('watertight', m_mesh.is_watertight())
                            print(f"alpha={alpha_v:.3f}")
                            print('edge manifold boundary', m_mesh.is_edge_manifold(allow_boundary_edges=False))

                        if m_mesh.is_watertight():
                            break
                if is_debug:
                    print(cell_key, '=======mesh info=========', m_mesh)
                    print('edge manifold', m_mesh.is_edge_manifold(allow_boundary_edges=True))
                    print('edge manifold boundary', m_mesh.is_edge_manifold(allow_boundary_edges=False))
                    print('vertex manifold', m_mesh.is_vertex_manifold())
                    print('watertight', m_mesh.is_watertight())
                    print(f"alpha={alpha_v:.3f}")
                    print('volume====>', m_mesh.get_volume(), 'using  ', (volume == cell_key).sum())
                    print('surface area=======>', m_mesh.get_surface_area(), 'weighted ',
                          cell_mask.sum() * weight_surface)
                    # o3d.visualization.draw_geometries([m_mesh], mesh_show_back_face=True, mesh_show_wireframe=True,
                    #                                   window_name=str(cell_key))
                # -----------------can not get watertight cell anyway-----------------------------------
                if not m_mesh.is_watertight():
                    print('no watertight mesh even after 10 times generation!!!', file_name, cell_key, 'use point sum')
                    volume_dict[cell_key] = (volume == cell_key).sum()
                    surface_dict[cell_key] = cell_mask.sum() * weight_surface  # 1.154... is derived by 2/sqrt(3)
                    irregularity = (surface_dict[cell_key] ** (1 / 2) / volume_dict[cell_key] ** (1 / 3))
                    if is_debug:
                        print(irregularity)
                    if irregularity < 2.199085:
                        print('impossible small surface!!!!!!!!!!!', time_point, cell_key)
                    for (cell1, cell2) in cell_conatact_pair_renew:
                        idx = str(cell1) + '_' + str(cell2)
                        # idx_test=
                        if cell_key not in (cell1, cell2) or idx in contact_dict.keys():
                            continue
                        # --------------------contact-----------------------------------------
                        contact_dict[idx] = len(contact_points_dict[idx]) * weight_surface
                    continue
                else:  # watertight mesh
                    m_mesh = o3d.geometry.TriangleMesh(
                        o3d.utility.Vector3dVector(np.asarray(m_mesh.vertices).astype(int)), m_mesh.triangles)
                    volume_dict[cell_key] = (volume == cell_key).sum()
                    surface_dict[cell_key] = m_mesh.get_surface_area()

                    if (surface_dict[cell_key] ** (1 / 2) / volume_dict[cell_key] ** (1 / 3)) < 2.199085:
                        print('impossible small surface', time_point, cell_key)

                    # if is_debug:
                    #     o3d.visualization.draw_geometries([m_mesh], mesh_show_back_face=True,mesh_show_wireframe=True)
                    #     print()
                    # ============contact surface detection========================
                    # cell_vertices = np.asarray(m_mesh.vertices).astype(int)
                    cell_vertices = np.asarray(m_mesh.vertices)
                    # ====================contact file ============================

                    for (cell1, cell2) in cell_conatact_pair_renew:
                        idx = str(cell1) + '_' + str(cell2)
                        # idx_test=
                        if cell_key not in (cell1, cell2) or idx in contact_dict.keys():
                            continue
                        # --------------------contact-----------------------------------------
                        # print('calculating, saving or showing',path_tmp, file_name, idx, ' contact surface')

                        # build a mask to erase not contact points
                        # enumerate each points in contact surface
                        contact_mask_not = [True for i in range(len(cell_vertices))]
                        contact_vertices_loc_list = []
                        for [x, y, z] in contact_points_dict[idx]:
                            # print(x,y,z)
                            contact_vertices_loc = np.where(np.prod(cell_vertices == [x, y, z], axis=-1))
                            if len(contact_vertices_loc[0]) != 0:
                                contact_vertices_loc_list.append(contact_vertices_loc[0][0])
                                contact_mask_not[contact_vertices_loc[0][0]] = False
                        contact_mesh_dict[idx] = contact_vertices_loc_list
                        contact_mesh = deepcopy(m_mesh)
                        contact_mesh.remove_vertices_by_mask(contact_mask_not)
                        # eodo: finished: 1. check 179 embryo,2. contact sum compare with cell surface area
                        alpha_surface_area = contact_mesh.get_surface_area()
                        points_surface_area = len(contact_points_dict[idx]) * weight_surface
                        contact_dict[
                            idx] = alpha_surface_area if alpha_surface_area > points_surface_area else points_surface_area

                        if is_debug:
                            # contact_mesh = deepcopy(m_mesh)
                            # contact_mesh.remove_vertices_by_mask(contact_mask_not)
                            print(idx, '=======mesh info=========', contact_mesh)
                            print('edge manifold', contact_mesh.is_edge_manifold(allow_boundary_edges=True))
                            print('edge manifold boundary', contact_mesh.is_edge_manifold(allow_boundary_edges=False))
                            print('vertex manifold', contact_mesh.is_vertex_manifold())
                            # print('watertight', m_mesh.is_watertight())
                            print('contact surface area=======>', contact_mesh.get_surface_area(), 'while points num',
                                  points_surface_area)
                            # o3d.visualization.draw_geometries([contact_mesh], mesh_show_back_face=True,
                            #                                   mesh_show_wireframe=True)
                # -----------------CHECK IF THE SUM OF CONTACT SURFACE AREA-------------------------

                if is_debug:
                    this_cell_surface = surface_dict[cell_key]
                    contact_list_tmp = []

                    for (cell1, cell2) in cell_conatact_pair_renew:
                        idx_tmp = str(cell1) + '_' + str(cell2)
                        # idx_test=
                        if cell_key not in (cell1, cell2):
                            continue
                        contact_list_tmp.append(contact_dict[idx_tmp])
                    print('contact sum', sum(contact_list_tmp), '  surface area ', this_cell_surface)
                    ratio_this_cell = sum(contact_list_tmp) / this_cell_surface
                    if ratio_this_cell > 1:
                        count_ratio_tmp.append(ratio_this_cell)
                        # print('impossible things!!!!')

    if is_debug:
        abnormal_contact_cell_ratio = len(count_ratio_tmp) / (len(cell_list) - 1)
        print('volume dict ', volume_dict)
        print('surface dict', surface_dict)
        print('contact dict', contact_dict)
        print('-------------abnormal contact cell ratio!!!!====>  ', abnormal_contact_cell_ratio)
    # ------------saving volume surface and contact file for an embryo------------
    else:
        path_tmp = os.path.join(stat_data_path, embryo_name)
        if not os.path.exists(path_tmp):
            os.mkdir(path_tmp)
        with open(os.path.join(path_tmp, file_name.split('.')[0] + '_volume.txt'), 'wb+') as handle:
            pickle.dump(volume_dict, handle, protocol=4)
        with open(os.path.join(path_tmp, file_name.split('.')[0] + '_surface.txt'), 'wb+') as handle:
            pickle.dump(surface_dict, handle, protocol=4)
        with open(os.path.join(path_tmp, file_name.split('.')[0] + '_contact.txt'), 'wb+') as handle:
            pickle.dump(contact_dict, handle, protocol=4)

    # -------------------------------------------------------------------------------------------------------


### Start go run with 12 processes to calculate the cell volume, cell suface and cell-cell contact area

In [3]:
max_times = [180, 187]
embryo_names = ['200109plc1p1', '200113plc1p2']


for idx, embryo_name in enumerate(embryo_names):
    configs = []
    config_tmp = {}
    config_tmp["embryo_name"] = embryo_name
    config_tmp['is_calculate_cell_mesh'] = True
    config_tmp['is_calculate_contact_file'] = True
    config_tmp['showCellMesh'] = False
    config_tmp['showCellContact'] = False
    config_tmp['path_embryo'] =os.path.join(r'./dataset/run', embryo_name, 'SegCellDividingCells')
    for tp in tqdm(range(1, max_times[idx] + 1), desc="Compose configs"):
        config_tmp['time_point'] = tp
        configs.append(config_tmp.copy())
    mpPool = mp.Pool(30)
#     mpPool = mp.Pool(12)

    for idx_, _ in enumerate(
            tqdm(mpPool.imap_unordered(calculate_cell_surface_and_contact_points, configs), total=max_times[idx],
                 desc="calculating {} segmentations (contact, surface, volume)".format(embryo_name))):
        #
        pass

Compose configs: 100%|█████████████████████████████████████| 180/180 [00:00<00:00, 382459.33it/s]
calculating 200109plc1p1 segmentations (contact, surface, volume): 100%|█| 180/180 [48:07:47<00:0
Compose configs: 100%|█████████████████████████████████████| 187/187 [00:00<00:00, 971914.31it/s]
calculating 200113plc1p2 segmentations (contact, surface, volume): 100%|█| 187/187 [46:52:31<00:0


## After a long time generating morphological features, we need to generate them as well as the cell position according to the x y z resolutions, raw image sizes and CD file location.

In [12]:
import io

from utils.data_structure import construct_celltree

max_times = [180, 187]
embryo_names = ['200109plc1p1', '200113plc1p2']

number_dictionary_path = os.path.join('./dataset', 'name_dictionary.csv')
pd_name_dict = pd.read_csv(number_dictionary_path, index_col=0, header=0)
label_name = pd_name_dict.to_dict()['0']
name_label = pd.Series(pd_name_dict.index, index=pd_name_dict['0']).to_dict()

# ----really important, ({raw image x}/{3D volume x}*{xy_resolution})-----------
volume_coefficient = ((512 / 256) * 0.09) ** 3
surface_coefficient = ((512 / 256) * 0.09) ** 2

for idx, embryo_name in enumerate(embryo_names):
    # ---------------------read cd file---------------------------------
    cd_file_path = os.path.join('./dataset/run/CDFiles','CD{}.csv'.format(embryo_name))
    cd_file_all_dict = {}
    with io.open(cd_file_path, mode="r", encoding="utf-8") as f:
        # next(f)
        next(f)
        for line in f:
            info_list = line.split(',')
            [cellname_, tp_] = info_list[0].split(':')
            cd_file_all_dict[(cellname_, tp_)] = info_list[1:]
    print('constructing cell tree for ', embryo_name)
    cell_tree,_=construct_celltree(cd_file_path,max_time=max_times[idx])
    # ---------------------------------------------------------------
    for tp in tqdm(range(1, max_times[idx] + 1), desc='generating {} nucloc file'.format(embryo_name)):
        cd_dict_this = {}
        no_repeat_record_list=[]
        for (cell_name_tem, tp_tem), values_tem in cd_file_all_dict.items():
            # print(type(tp_tem),type(tp))
            if str(tp) == tp_tem:
                cd_dict_this[cell_name_tem] = values_tem

        path_tmp = os.path.join(stat_data_path,embryo_name)
        # read the cell volume pkl file 
        with open(os.path.join(path_tmp, '{}_{}_segCell_volume.txt'.format(embryo_name, str(tp).zfill(3))),
                  'rb') as handle:
            volume_dict = pickle.load(handle)
        # read the cell surface area pkl file
        with open(os.path.join(path_tmp, '{}_{}_segCell_surface.txt'.format(embryo_name, str(tp).zfill(3))),
                  'rb') as handle:
            surface_dict = pickle.load(handle)
        # x y z is the size of the 3D volume, designed by you
        df_nucLoc = pd.DataFrame(
            columns=['nucleus_label','nucleus_name','x_256','y_356','z_214','volume','surface','note'])
        for cell_name_, zxy_pos in cd_dict_this.items():
            cell_label_ = name_label[cell_name_]
#             print(zxy_pos)
            if cell_label_ in volume_dict.keys():
                y = int(float(zxy_pos[8]) / 712 * 356)# labelled as X in cd file
                x = int(float(zxy_pos[9]) / 512 * 256)# labelled as Y in cd file
                z = 214 - int(float(zxy_pos[7]) / 92 * 214)

                df_nucLoc.loc[len(df_nucLoc.index)] = [cell_label_, cell_name_, x, y, z,
                                                       volume_dict[cell_label_]*volume_coefficient, surface_dict[cell_label_]*surface_coefficient, None]
                continue
            # -------------deal with dividing or lost cellsssssssssssssss----------------------------
            cell_mother_node = cell_tree.parent(cell_name_)
            cell_mother_name_=cell_mother_node.tag
            cell_mother_label_ = name_label.get(cell_mother_name_, None)
            if cell_mother_label_ in volume_dict.keys() and cell_mother_label_ not in no_repeat_record_list:
                no_repeat_record_list.append(cell_mother_label_)

                children_list=cell_tree.children(cell_mother_name_)
                daughter_cell1=children_list[0].tag
                daughter_cell2=children_list[1].tag
                print('dividing ', embryo_name, tp, cell_mother_name_,daughter_cell1,daughter_cell2)

                cell_label1=name_label[daughter_cell1]
                cell_label2=name_label[daughter_cell2]

                zxy_pos1 = cd_dict_this[daughter_cell1]
                zxy_pos2 = cd_dict_this[daughter_cell2]
                
#                 print(zxy_pos1,zxy_pos2)

                y1 = int(float(zxy_pos1[8]) / 712 * 356)# labelled as X in cd file
                x1 = int(float(zxy_pos1[9]) / 512 * 256)# labelled as Y in cd file
                z1 = 214 - int(float(zxy_pos1[7]) / 92 * 214)

                y2 = int(float(zxy_pos2[8]) / 712 * 356)# labelled as X in cd file
                x2 = int(float(zxy_pos2[9]) / 512 * 256)# labelled as Y in cd file
                z2 = 214 - int(float(zxy_pos2[7]) / 92 * 214)

                df_nucLoc.loc[len(df_nucLoc.index)] = [cell_mother_label_, cell_mother_name_, None, None, None,
                                                       volume_dict[cell_mother_label_]*volume_coefficient,
                                                       surface_dict[cell_mother_label_]*surface_coefficient, 'mother']
                df_nucLoc.loc[len(df_nucLoc.index)] = [cell_label1, daughter_cell1, x1,y1, z1,None,None,
                                                       'child of {}'.format(cell_mother_name_)]
                df_nucLoc.loc[len(df_nucLoc.index)] = [cell_label2, daughter_cell2, x2,y2, z2,None,None,
                                                       'child of {}'.format(cell_mother_name_)]
            elif cell_mother_label_ not in no_repeat_record_list:
                print('lost cell ', embryo_name, tp,cell_name_)
                y = int(float(zxy_pos[8]) / 712 * 356)  # labelled as X in cd file
                x = int(float(zxy_pos[9]) / 512 * 256)  # labelled as Y in cd file
                z = 214 - int(float(zxy_pos[7]) / 92 * 214)

                df_nucLoc.loc[len(df_nucLoc.index)] = [cell_label_, cell_name_, x, y, z,
                                                       None, None, 'lost']

        nucLoc_save_file_path = os.path.join(stat_data_path, 'NucLocFile', embryo_name,
                                             '{}_{}_nucLoc.csv'.format(embryo_name,
                                                                       str(tp).zfill(3)))
        df_nucLoc.to_csv(nucLoc_save_file_path, index=False)

constructing cell tree for  200109plc1p1


generating 200109plc1p1 nucloc file:   2%|▌                      | 4/180 [00:00<00:10, 16.73it/s]

dividing  200109plc1p1 2 ABa ABal ABar
dividing  200109plc1p1 2 ABp ABpl ABpr
dividing  200109plc1p1 3 ABa ABal ABar
dividing  200109plc1p1 3 ABp ABpl ABpr
dividing  200109plc1p1 4 ABp ABpl ABpr
dividing  200109plc1p1 5 EMS E MS


generating 200109plc1p1 nucloc file:   4%|█                      | 8/180 [00:00<00:09, 17.66it/s]

dividing  200109plc1p1 6 EMS E MS
dividing  200109plc1p1 8 P2 P3 C
dividing  200109plc1p1 9 P2 P3 C


...

dividing  200113plc1p2 186 ABalpapppa ABalpapppaa ABalpapppap
dividing  200113plc1p2 186 ABplaaaaaa ABplaaaaaaa ABplaaaaaap


generating 200113plc1p2 nucloc file:  99%|████████████████████▉| 186/187 [01:11<00:01,  1.10s/it]

dividing  200113plc1p2 186 Cappaa Cappaaa Cappaap
dividing  200113plc1p2 187 ABalaapaap ABalaapaapa ABalaapaapp
dividing  200113plc1p2 187 ABarpaaaap ABarpaaaapa ABarpaaaapp
dividing  200113plc1p2 187 ABprppaapa ABprppaapaa ABprppaapap


generating 200113plc1p2 nucloc file: 100%|█████████████████████| 187/187 [01:12<00:00,  2.57it/s]

dividing  200113plc1p2 187 Cppppa Cppppaa Cppppap


## After we get the nucleus location files, we can assemble all the statistical data at different time point in one csv file for generating visulized GUI Data. 

In [16]:
from treelib import Tree

vol_coefficient = ((512 / 256) * 0.09) ** 3
sur_coefficient = ((512 / 256) * 0.09) ** 2

# remember to adjust your parameters here, and create the dirtectory ./middle_output/statistics/{embryo name}

for idx, embryo_name in enumerate(embryo_names):
    cd_file_path = os.path.join('dataset/run/CDFiles', 'CD{}.csv'.format(embryo_name))
    cell_tree, max_time = construct_celltree(cd_file_path, max_times[idx])

    # ------------surface-------------and-----------volume-----------------------
    all_names = [cname for cname in cell_tree.expand_tree(mode=Tree.WIDTH)]
    # for idx, cell_name in enumerate(all_names):
    volume_embryo = pd.DataFrame(
        np.full(shape=(max_time, len(all_names)), fill_value=np.nan, dtype=np.float32),
        index=range(1, max_time + 1), columns=all_names)

    surface_embryo = pd.DataFrame(
        np.full(shape=(max_time, len(all_names)), fill_value=np.nan, dtype=np.float32),
        index=range(1, max_time + 1), columns=all_names)

    for tp in tqdm(range(1, max_time + 1),
                   desc='assembling volume and surface area of {} result'.format(embryo_name)):
        path_tmp = os.path.join(stat_data_path, embryo_name)
        with open(os.path.join(path_tmp, '{}_{}_segCell_volume.txt'.format(embryo_name, str(tp).zfill(3))),
                  'rb') as handle:
            volume_dict = pickle.load(handle)
        with open(os.path.join(path_tmp, '{}_{}_segCell_surface.txt'.format(embryo_name, str(tp).zfill(3))),
                  'rb') as handle:
            surface_dict = pickle.load(handle)

        for cell_label_, vol_value in volume_dict.items():
            cell_name_ = label_name[cell_label_]
            volume_embryo.loc[tp, cell_name_] = vol_value * vol_coefficient

        for cell_label_, sur_value in surface_dict.items():
            cell_name_ = label_name[cell_label_]
            surface_embryo.loc[tp, cell_name_] = sur_value * sur_coefficient

    volume_embryo = volume_embryo.loc[:, ((volume_embryo != 0) & (~np.isnan(volume_embryo))).any(axis=0)]
    volume_embryo.to_csv(os.path.join(stat_data_path, 'statistics', embryo_name, embryo_name + '_volume.csv'))
    surface_embryo = surface_embryo.loc[:, ((surface_embryo != 0) & (~np.isnan(surface_embryo))).any(axis=0)]
    surface_embryo.to_csv(os.path.join(stat_data_path, 'statistics', embryo_name, embryo_name + '_surface.csv'))

    # ------------contact-----------initialize the contact csv  file----------------------
    # Get tuble lists with elements from the list

    name_combination = []
    first_level_names = []
    for i, name1 in enumerate(all_names):
        for name2 in all_names[i + 1:]:
            if not (cell_tree.is_ancestor(name1, name2) or cell_tree.is_ancestor(name2, name1)):
                first_level_names.append(name1)
                name_combination.append((name1, name2))

    multi_index = pd.MultiIndex.from_tuples(name_combination, names=['cell1', 'cell2'])
    # print(multi_index)
    stat_embryo = pd.DataFrame(
        np.full(shape=(max_time, len(name_combination)), fill_value=np.nan, dtype=np.float32),
        index=range(1, max_time + 1), columns=multi_index)
    # set zero element to express the exist of the specific nucleus
    for cell_name in all_names:
        if cell_name not in first_level_names:
            continue
        try:
            cell_time = cell_tree.get_node(cell_name).data.get_time()
            cell_time = [x for x in cell_time if x <= max_time]
            stat_embryo.loc[cell_time, (cell_name, slice(None))] = 0
        except:
            cell_name
    # print(stat_embryo)
    # edges_view = point_embryo.edges(data=True)
    for tp in tqdm(range(1, max_times[idx] + 1),
                   desc='assembling contact surface of {} result'.format(embryo_name)):
        path_tmp = os.path.join(stat_data_path, embryo_name)
        with open(os.path.join(path_tmp, '{}_{}_segCell_contact.txt'.format(embryo_name, str(tp).zfill(3))),
                  'rb') as handle:
            contact_dict = pickle.load(handle)
        for contact_sur_idx, contact_sur_value in contact_dict.items():
            [cell1, cell2] = contact_sur_idx.split('_')
            cell1_name = label_name[int(cell1)]
            cell2_name = label_name[int(cell2)]
            if (cell1_name, cell2_name) in stat_embryo.columns:
                stat_embryo.loc[tp, (cell1_name, cell2_name)] = contact_sur_value * sur_coefficient
            elif (cell2_name, cell1_name) in stat_embryo.columns:
                stat_embryo.loc[tp, (cell2_name, cell1_name)] = contact_sur_value * sur_coefficient
            else:
                pass
                # print('columns missing (cell1_name, cell2_name)')
    stat_embryo = stat_embryo.loc[:, ((stat_embryo != 0) & (~np.isnan(stat_embryo))).any(axis=0)]
    print(stat_embryo)
    stat_embryo.to_csv(os.path.join(stat_data_path, 'statistics', embryo_name, embryo_name + '_contact.csv'))
    # --------------------------------------------------------------------------------------------

assembling volume and surface area of 200109plc1p1 result: 100%|█| 180/180 [00:05<00:00, 34.93it/
assembling contact surface of 200109plc1p1 result: 100%|███████| 180/180 [39:02<00:00, 13.02s/it]


cell1         ABa                     ABp                                     \
cell2         ABp         EMS         EMS          P2       ABal        ABar   
1      336.596099  352.110315  322.602121  291.183887   0.000000    0.000000   
2      325.291772  319.678588  301.279821  266.158444        NaN         NaN   
3      314.806033  281.945523  305.489708  241.171982        NaN         NaN   
4             NaN         NaN  347.861447  258.752161  94.449606  211.546848   
5             NaN         NaN         NaN         NaN        NaN         NaN   
..            ...         ...         ...         ...        ...         ...   
176           NaN         NaN         NaN         NaN        NaN         NaN   
177           NaN         NaN         NaN         NaN        NaN         NaN   
178           NaN         NaN         NaN         NaN        NaN         NaN   
179           NaN         NaN         NaN         NaN        NaN         NaN   
180           NaN         NaN         Na

assembling volume and surface area of 200113plc1p2 result: 100%|█| 187/187 [00:04<00:00, 41.79it/
assembling contact surface of 200113plc1p2 result: 100%|███████| 187/187 [41:04<00:00, 13.18s/it]


cell1          AB          P1                     ABa       ABp            \
cell2          P1         ABa         ABp         ABp  EMS  EMS   P2 ABal   
1      436.970732    0.000000    0.000000         NaN  NaN  NaN  NaN  NaN   
2      382.593019    0.000000    0.000000    0.000000  0.0  0.0  0.0  0.0   
3      426.718877    0.000000    0.000000    0.000000  0.0  0.0  0.0  0.0   
4             NaN  215.172029  264.248403  253.411840  0.0  0.0  0.0  0.0   
5             NaN  266.548249  396.080251  214.704264  0.0  0.0  0.0  0.0   
..            ...         ...         ...         ...  ...  ...  ...  ...   
183           NaN         NaN         NaN         NaN  NaN  NaN  NaN  NaN   
184           NaN         NaN         NaN         NaN  NaN  NaN  NaN  NaN   
185           NaN         NaN         NaN         NaN  NaN  NaN  NaN  NaN   
186           NaN         NaN         NaN         NaN  NaN  NaN  NaN  NaN   
187           NaN         NaN         NaN         NaN  NaN  NaN  NaN  NaN   